In [40]:
import os, re, glob
train_path = "G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment"
musan_path = "G:/SP_CUP_2024/data_augmentation/musan"
rir_path = "G:/SP_CUP_2024/data_augmentation/RIRS_NOISES/simulated_rirs"
libri_train_path = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/train"
libri_test_path = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/test"
validation_path = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/dev"
model_path = "G:/SP_CUP_2024/model/20240203"
num_frames = 100
batch_size = 100
def extract_number(filename):
    match = re.search(r'spk_(\d+)-', filename)
    if match:
        return int(match.group(1))
    else:
        return float('inf')  # 일치하는 부분이 없는 경우 맨 뒤로 정렬
#single channel enrollment train_list 
file_list = os.listdir(train_path)
train_files = glob.glob(os.path.join(train_path,'*.wav').replace("\\", "/"))
with open("G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment/annotations.txt", "w") as train_list:
    for files in train_files:
        file_name = files
        labels = extract_number(files)
        train_list.write(str(labels) +" " + file_name + "\n")
train_list = "G:/SP_CUP_2024/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment/annotations.txt"

#librispeech test_list
libri_test_files = glob.glob(os.path.join(libri_test_path,'*/*/*/*.wav').replace("\\", "/"))
with open("G:/SP_CUP_2024/data_augmentation/mls_french_opus/test/annotations.txt", "w") as libri_test_list:
    for files in libri_test_files:
        file_name = files
        labels = files.split('\\')[2]
        libri_test_list.write(str(labels) +" " + file_name + "\n")
libri_test_list = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/test/annotations.txt"

#librispeech train_list
libri_train_files = glob.glob(os.path.join(libri_train_path,'*/*/*/*.wav').replace("\\", "/"))
label_files = {}
for file in libri_train_files:
    label = file.split('\\')[2]
    if label not in label_files:
        label_files[label] = []
    label_files[label].append(file)
with open("G:/SP_CUP_2024/data_augmentation/mls_french_opus/train/annotations.txt", "w") as libri_train_list:
    for label, files in label_files.items():
        for i in range(10):
            for file in files[:10]:
                file_name = file
                labels = label
                libri_train_list.write(str(labels) +" " + file_name + "\n")
libri_train_list = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/train/annotations.txt"

#librispeech validation_list
validation_files = glob.glob(os.path.join(validation_path,'*/*/*/*.wav'))
with open("G:/SP_CUP_2024/data_augmentation/mls_french_opus/dev/annotations.txt", "w") as validation_list:
    for label, files in label_files.items():
        for file in files[10:12]:
            file_name = file
            labels = label
            validation_list.write(str(labels) +" " + file_name + "\n")
validation_list = "G:/SP_CUP_2024/data_augmentation/mls_french_opus/dev/annotations.txt"

In [41]:
'''
DataLoader for training
'''

import glob, numpy, os, random, soundfile, torch
from scipy import signal
import librosa

class train_loader(object):
    def __init__(self, file_list, file_path, musan_path, rir_path, num_frames, **kwargs):
        self.train_path = train_path
        self.num_frames = num_frames
        # Load and configure augmentation files
        self.noisetypes = ['noise','speech','music']
        self.noisesnr = {'noise':[0,15],'speech':[13,20],'music':[5,15]}
        self.numnoise = {'noise':[1,1], 'speech':[3,8], 'music':[1,1]}
        self.noiselist = {}
        augment_files   = glob.glob(os.path.join(musan_path,'*/*/*.wav').replace("\\", "/"))
        for file in augment_files:
            if file.split('\\')[-3] not in self.noiselist:
                self.noiselist[file.split('\\')[-3]] = []
            self.noiselist[file.split('\\')[-3]].append(file)
        self.rir_files  = glob.glob(os.path.join(rir_path,'*/*/*.wav').replace("\\", "/"))
        
        # Load data & labels
        self.data_list  = []
        self.data_label = []
        lines = open(file_list).read().splitlines()
        dictkeys = list(set([x.split()[0] for x in lines]))
        dictkeys.sort()
        dictkeys = { key : ii for ii, key in enumerate(dictkeys) }
        for index, line in enumerate(lines):
            speaker_label = dictkeys[line.split()[0]]
            file_name = line.split()[1]
            self.data_label.append(speaker_label)
            self.data_list.append(file_name)
        print(self.data_label)
        print(len(self.data_label))
        
    def __len__(self):
        return len(self.data_list)

    def add_rev(self, audio):
        rir_file    = random.choice(self.rir_files)
        rir, sr     = librosa.load(rir_file,sr = 16000)
        rir         = numpy.expand_dims(rir.astype(numpy.float),0)
        rir         = rir / numpy.sqrt(numpy.sum(rir**2))
        return signal.convolve(audio, rir, mode='full')[:,:self.num_frames * 160 + 240]

    def add_noise(self, audio, noisecat):
        clean_db    = 10 * numpy.log10(numpy.mean(audio ** 2)+1e-4) 
        numnoise    = self.numnoise[noisecat]
        noiselist   = random.sample(self.noiselist[noisecat], random.randint(numnoise[0],numnoise[1]))
        noises = []
        for noise in noiselist:
            noiseaudio, sr = librosa.load(noise,sr = 16000)
            length = self.num_frames * 160 + 240
            if noiseaudio.shape[0] <= length:
                shortage = length - noiseaudio.shape[0]
                noiseaudio = numpy.pad(noiseaudio, (0, shortage), 'wrap')
            start_frame = numpy.int64(random.random()*(noiseaudio.shape[0]-length))
            noiseaudio = noiseaudio[start_frame:start_frame + length]
            noiseaudio = numpy.stack([noiseaudio],axis=0)
            noise_db = 10 * numpy.log10(numpy.mean(noiseaudio ** 2)+1e-4) 
            noisesnr   = random.uniform(self.noisesnr[noisecat][0],self.noisesnr[noisecat][1])
            noises.append(numpy.sqrt(10 ** ((clean_db - noise_db - noisesnr) / 10)) * noiseaudio)
        noise = numpy.sum(numpy.concatenate(noises,axis=0),axis=0,keepdims=True)
        return noise + audio
    
    def __getitem__(self, index):
        # Read the utterance and randomly select the segment
        audio, sr = librosa.load(self.data_list[index],sr = 16000)
        length = self.num_frames * 160 + 240
        if audio.shape[0] <= length:
            shortage = length - audio.shape[0]
            audio = numpy.pad(audio, (0, shortage), 'wrap') #오디오 길이가 2초보다 짧을 때는 wrapping
        start_frame = numpy.int64(random.random()*(audio.shape[0]-length))
        audio = audio[start_frame:start_frame + length]
        audio = numpy.stack([audio],axis=0) #걍 (1, length)형태로 만들어주기 위한 stack 없으면 ( , length)니께
        # Data Augmentation
        augtype = random.randint(0,3)
        if augtype == 0:   # Original
            audio = audio
        elif augtype == 1: # Reverberation
            audio = self.add_rev(audio)
#         elif augtype == 2: # Babble
#             audio = self.add_noise(audio, 'speech')
#         elif augtype == 3: # Music
#             audio = self.add_noise(audio, 'music')
        elif augtype == 2: # Noise
            audio = self.add_noise(audio, 'noise')
        elif augtype == 3: # Noise + 잔향
            audio = self.add_noise(audio, 'noise')
            audio = self.add_rev(audio)
#         elif augtype == 5: # Television noise
#             audio = self.add_noise(audio, 'speech')
#             audio = self.add_noise(audio, 'music')
        return torch.FloatTensor(audio[0]), self.data_label[index]

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class LayerNorm(nn.Module): # ⚡
    """ LayerNorm that supports two data formats: channels_last (default) or channels_first. 
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with 
    shape (batch_size, T, channels) while channels_first corresponds to inputs 
    with shape (batch_size, channels, T).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None] * x + self.bias[:, None] # ⚡
            return x

class GRN(nn.Module):  # ⚡
    """ GRN (Global Response Normalization) layer
    """
    def __init__(self, dim):
        super().__init__()
        self.gamma = nn.Parameter(torch.zeros(1, 1, dim))
        self.beta = nn.Parameter(torch.zeros(1, 1, dim))

    def forward(self, x):
        Gx = torch.norm(x, p=2, dim=1, keepdim=True)
        Nx = Gx / (Gx.mean(dim=-1, keepdim=True) + 1e-6)
        return self.gamma * (x * Nx) + self.beta + x

In [44]:
import numpy as np
import librosa
import torch
import torch.nn as nn
import torchaudio
import torch.nn.functional as F
from torchaudio.transforms import MFCC, MelSpectrogram
from torch import optim
from sklearn.metrics import accuracy_score

class TDNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TDNN, self).__init__()
        self.dim = 256
        self.tdnn1 = nn.Sequential(
            nn.Conv1d(input_dim, 256, kernel_size=4),
            nn.BatchNorm1d(256),
            nn.GELU()
        )          
        self.tdnn2 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, dilation=2),
            nn.BatchNorm1d(256),
            nn.GELU()
        )
        self.tdnn3 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, dilation=3),
            nn.BatchNorm1d(256),
            nn.GELU()
        )
        
        self.dwconv = nn.Conv1d(self.dim, self.dim, kernel_size=7, padding=3, groups=self.dim) # depthwise conv 
        self.norm = LayerNorm(self.dim, eps=1e-6)
        self.pwconv1 = nn.Linear(self.dim, 4 * self.dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.grn = GRN(4 * self.dim)
        self.pwconv2 = nn.Linear(4 * self.dim, self.dim)
        self.drop_path = nn.Identity()
        
        self.segment6 = nn.Linear(198, 256)
        self.segment7 = nn.Linear(256, 256)
        self.output = nn.Linear(256, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x1 = self.tdnn1(x)
        x2 = self.tdnn2(x1)
        x3 = self.tdnn3(x2)
        
        x2_resized = F.interpolate(x2, size=x1.size()[2], mode='nearest')
        x3_resized = F.interpolate(x3, size=x1.size()[2], mode='nearest')        

        x = x1 + x2_resized + x3_resized
        inputs = x
        x = self.dwconv(x)
        x = x + inputs
        inputs = x
        x = x.permute(0, 2, 1) # (N, C, T) -> (N, T, C) # ⚡
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.grn(x)
        x = self.pwconv2(x)
        x = x.permute(0, 2, 1) # (N, T, C) -> (N, C, T) # ⚡
        

        x = inputs + self.drop_path(x)
        mean = torch.mean(x,1)
        std = torch.var(x,1)
        stat_pooling = torch.cat((mean,std),1)
        x_vec = self.segment6(stat_pooling)
        segment7_out = self.segment7(x_vec)
        predictions = self.output(segment7_out)
        return predictions,x_vec
#       stats = torch.cat((x.mean(dim=2), x.std(dim=2)), dim=1)
#       print(stats.shape)
#       x = self.fc(stats)
#       return x, stats

# 모델 로드 함수 정의
def load_model(model_path, device):
    checkpoint = torch.load(model_path +'/'+ 'best_check_point_199_9.2916.pth', map_location=device)
    model = TDNN(input_dim=120, num_classes=142).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.002, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
    
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model,optimizer

# MFCC 추출
trainloader = train_loader(libri_train_list, libri_train_path, musan_path, rir_path, num_frames)
trainLoader = torch.utils.data.DataLoader(trainloader, batch_size = batch_size, shuffle = True, drop_last = True)

validationloader = train_loader(validation_list, validation_path, musan_path, rir_path, num_frames)
validationLoader = torch.utils.data.DataLoader(validationloader, batch_size = batch_size, shuffle = True, drop_last = True)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
# model = TDNN(input_dim=120, num_classes=142).to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.002, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
model,optimizer = load_model(model_path, device)

loss_fun = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma=0.99)

# train_features, train_labels = next(iter(trainLoader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# print(train_labels)
num_epochs = 10000  # 예시 epoch 수
import torch
import torchaudio
import torchaudio.transforms as T

def preprocess_audio(signal, sample_rate=16000, n_filterbanks=24, context_frames=2, mean_normalize=True):
    # 필터뱅크 변환 설정
    n_fft = int(sample_rate * 0.025)  # 25ms window
    hop_length = int(sample_rate * 0.01)  # 10ms hop
    mel_transform = T.MelSpectrogram(sample_rate=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mels=n_filterbanks)
    fbank = mel_transform(signal)
    
    if mean_normalize:
        cmn_window = int(3 * sample_rate / hop_length)  # 3초의 슬라이딩 윈도우
        cmn_transform = T.SlidingWindowCmn(cmn_window=cmn_window, min_cmn_window=1, center=True)
        fbank = cmn_transform(fbank)

    # [t-2,t+2]stacking
    padded = F.pad(fbank, (context_frames, context_frames), "constant", 0)
    sequences = [padded[:, :, i:(i + fbank.size(2))] for i in range(2 * context_frames + 1)]
    stacked_features = torch.cat(sequences, dim=1)  # 차원 증가
    return stacked_features

def train(trainLoader,epoch):
    train_loss_list=[]
    full_preds=[]
    full_gts=[]
    model.train()
    for data in trainLoader:
        inputs, labels = data
        mfccs = preprocess_audio(inputs).cuda()
        features = torch.tensor(mfccs, dtype=torch.float32)
        labels_tensor = torch.tensor(labels, dtype=torch.long)
        features , labels_tensor = features.to(device), labels_tensor.to(device)
        features.requires_grad = True
        optimizer.zero_grad()
        pred_logits,x_vec = model(features)
        #### CE loss
        loss = loss_fun(pred_logits,labels_tensor)
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
        #train_acc_list.append(accuracy)
        #if i_batch%10==0:
        #    print('Loss {} after {} iteration'.format(np.mean(np.asarray(train_loss_list)),i_batch))

        predictions = np.argmax(pred_logits.detach().cpu().numpy(),axis=1)
        for pred in predictions:
            full_preds.append(pred)
        for lab in labels_tensor.detach().cpu().numpy():
            full_gts.append(lab)
    
    scheduler.step(epoch)
    mean_acc = accuracy_score(full_gts,full_preds)
    mean_loss = np.mean(np.asarray(train_loss_list))
    print('Total training loss {} and training Accuracy {} after {} epochs'.format(mean_loss,mean_acc,epoch))
    
#     model_save_path = os.path.join('G:/SP_CUP_2024/model/20240121/', 'best_check_point_'+str(epoch)+'_'+str(mean_loss))
    


def validation(validationLoader,epoch):
    model.eval()
    with torch.no_grad():
        val_loss_list=[]
        full_preds=[]
        full_gts=[]
        for data in validationLoader:
            inputs, labels = data
            # batch_data에서 오디오 데이터 추출
            mfccs = preprocess_audio(inputs)
            features = torch.tensor(mfccs, dtype=torch.float32)
            labels_tensor = torch.tensor(labels, dtype=torch.long)
            features , labels_tensor = features.to(device), labels_tensor.to(device)
            pred_logits,x_vec = model(features)
            #### CE loss
            loss = loss_fun(pred_logits,labels_tensor)
            val_loss_list.append(loss.item())
            #train_acc_list.append(accuracy)
            predictions = np.argmax(pred_logits.detach().cpu().numpy(),axis=1)
            for pred in predictions:
                full_preds.append(pred)
            for lab in labels_tensor.detach().cpu().numpy():
                full_gts.append(lab)
                
        mean_acc = accuracy_score(full_gts,full_preds)
        mean_loss = np.mean(np.asarray(val_loss_list))
        print('Total vlidation loss {} and Validation accuracy {} after {} epochs'.format(mean_loss,mean_acc,epoch))
        
        model_save_path = os.path.join('G:/SP_CUP_2024/model/20240203/', f'best_check_point_{epoch}_{mean_loss:.4f}.pth')
        state_dict = {'model': model.state_dict(),'optimizer': optimizer.state_dict(),'epoch': epoch}
        torch.save(state_dict, model_save_path)
    



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54, 55, 55, 56, 56, 57, 57, 58, 58, 59, 59, 60, 60, 61, 61, 62, 62, 63, 63, 64, 64, 65, 65, 66, 66, 67, 67, 68, 68, 69, 69, 70, 70, 71, 71, 72, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 78, 78, 79, 79, 80, 80, 81, 81, 82, 82, 83, 83, 84, 84, 85, 85, 86, 86, 87, 87, 88, 88, 89, 89, 90, 90, 91, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 96, 97, 97, 98, 98, 99, 99, 100, 100, 101, 101, 102, 102, 103, 103, 104, 104]
210
cuda


In [45]:
if __name__ == '__main__':
    for epoch in range(num_epochs):
        print(epoch)
        train(trainLoader,epoch)
        validation(validationLoader,epoch)

0


C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\4243142822.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rir         = numpy.expand_dims(rir.astype(numpy.float),0)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\1858052848.py:141: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(mfccs, dtype=torch.float32)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\1858052848.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().de

Total training loss 3.1268339773019154 and training Accuracy 0.2165 after 0 epochs


C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\4243142822.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rir         = numpy.expand_dims(rir.astype(numpy.float),0)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\1858052848.py:181: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(mfccs, dtype=torch.float32)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\1858052848.py:182: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().de

Total vlidation loss 9.514750480651855 and Validation accuracy 0.015 after 0 epochs
1


...

Total training loss 1.71898339788119 and training Accuracy 0.55275 after 137 epochs


C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\4243142822.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rir         = numpy.expand_dims(rir.astype(numpy.float),0)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\1858052848.py:181: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(mfccs, dtype=torch.float32)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\1858052848.py:182: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().de

Total vlidation loss 11.220837593078613 and Validation accuracy 0.03 after 137 epochs
138


C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\4243142822.py:46: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rir         = numpy.expand_dims(rir.astype(numpy.float),0)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\1858052848.py:141: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(mfccs, dtype=torch.float32)
C:\Users\TCLAB-TEMP\AppData\Local\Temp\ipykernel_11208\1858052848.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().de

KeyboardInterrupt: 

In [ ]:
# torch.Size([100, 16240])
# torch.Size([100, 16240])
# torch.Size([100, 24, 102])
# cmn torch.Size([100, 24, 102])
# feature torch.Size([100, 120, 102])